# 1. Importación de paquetes

In [59]:
#Import packages
from modules import preprocess
from modules import training

# 2. Preprocesamiento del *dataset*: [Room Occupancy Estimation Data Set](https://www.kaggle.com/ananthr1/room-occupancy-estimation-data-set)

## Paso 1: Lectura del *dataset* y obtención de datos

In [60]:
df = preprocess.load_csv_data('Occupancy_Estimation.csv')
df.shape

(10129, 19)

In [61]:
df.head()

,Date,Time,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR,Room_Occupancy_Count
0,2017/12/22,10:49:41,24.94,24.75,24.56,25.38,121,34,53,40,0.08,0.19,0.06,0.06,390,0.769231,0,0,1
1,2017/12/22,10:50:12,24.94,24.75,24.56,25.44,121,33,53,40,0.93,0.05,0.06,0.06,390,0.646154,0,0,1
2,2017/12/22,10:50:42,25.00,24.75,24.50,25.44,121,34,53,40,0.43,0.11,0.08,0.06,390,0.519231,0,0,1
3,2017/12/22,10:51:13,25.00,24.75,24.56,25.44,121,34,53,40,0.41,0.10,0.10,0.09,390,0.388462,0,0,1
4,2017/12/22,10:51:44,25.00,24.75,24.56,25.44,121,34,54,40,0.18,0.06,0.06,0.06,390,0.253846,0,0,1


## Paso 2: Recodificación de la columna *RoomOccupancyCount*

En este caso, la columna que contiene la variable de salida (*RoomOccupancyCount*) cuenta el número de personas en la sala. Para simplificar, nuestro objetivo va a ser detectar si la sala está vacía o, en cambio, hay alguien (no el número exacto de personas).

Por lo tanto, se **recodificará cualquier valor mayor que 0 en esa columna como 1**, de tal forma que la variable de salida sea binaria:
- 0 -> ausencia.
- 1 -> presencia.

In [62]:
# Show "Room_Occupancy_Count" before processing
df["Room_Occupancy_Count"]

0        1
1        1
2        1
3        1
4        1
        ..
10124    0
10125    0
10126    0
10127    0
10128    0
Name: Room_Occupancy_Count, Length: 10129, dtype: int64

In [63]:
#Process "Room_Occupancy_Count"
preprocess.recode_dataset_output(df)

# Show "Room_Occupancy_Count" after processing
df["Room_Occupancy_Count"]

0        1
1        1
2        1
3        1
4        1
        ..
10124    0
10125    0
10126    0
10127    0
10128    0
Name: Room_Occupancy_Count, Length: 10129, dtype: int32

## Paso 3: Eliminación de las columnas temporales *Date* y *Time*

Respecto a los datos ofrecidos por las columas *Date* y *Time*, vamos a filtrarlos y dejarlos fuera del proceso. Ya que, aunque los incluyeramos (por ejemplo, juntando ambas partes en una sola columna y convirtiéndolo a formato 'epoch', como long int), los valores de muestras sucesivas de esa columna estarían totalmente correlados entre sí y, como consecuencia, fastidiaríamos a la mayoría de algoritmos que vamos a emplear posteriormente.

Se debe tener en cuenta que no nos estamos olvidando de la información temporal para considerar como si cada valor muestreado (fila de la tabla) fuese independiente de las demás filas. Sabemos que eso no es así, pero estamos considerando que esa información temporal no nos ofrece valor añadido para predecir si la habitación está ocupada o vacía. En caso contrario, se tendrían que usar modelos bastante más complicados para considerar esa relación temporal que indica que, en realidad, las muestras de dos filas adyacentes son consecutivas en el tiempo.

In [64]:
# Filter Date and Time columns
preprocess.remove_time_columns(df)

# Show the result
df.head()

,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR,Room_Occupancy_Count
0,24.94,24.75,24.56,25.38,121,34,53,40,0.08,0.19,0.06,0.06,390,0.769231,0,0,1
1,24.94,24.75,24.56,25.44,121,33,53,40,0.93,0.05,0.06,0.06,390,0.646154,0,0,1
2,25.00,24.75,24.50,25.44,121,34,53,40,0.43,0.11,0.08,0.06,390,0.519231,0,0,1
3,25.00,24.75,24.56,25.44,121,34,53,40,0.41,0.10,0.10,0.09,390,0.388462,0,0,1
4,25.00,24.75,24.56,25.44,121,34,54,40,0.18,0.06,0.06,0.06,390,0.253846,0,0,1


## Paso 4: División del *dataset* en *train* y *test*

In [65]:
# Split dataset into train and test
train_df, test_df = preprocess.split_dataset(df)
train_df.shape, test_df.shape

((7596, 17), (2533, 17))

In [66]:
train_df.head()

,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR,Room_Occupancy_Count
894,26.31,27.94,26.06,26.44,150,239,182,11,1.65,0.34,0.10,0.09,1210,0.426923,1,0,1
1165,25.69,25.69,25.31,25.88,0,0,0,0,0.07,0.05,0.06,0.06,620,-2.200000,0,0,0
249,26.00,27.88,25.81,26.19,164,253,270,70,0.15,0.14,0.07,0.07,860,0.880769,0,1,1
9011,25.31,25.31,24.94,25.38,0,0,0,0,0.06,0.05,0.05,0.09,345,0.000000,0,0,0
1659,25.38,25.31,24.88,25.75,0,0,0,0,0.07,0.05,0.06,0.06,370,0.000000,0,0,0


In [67]:
test_df.head()

,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR,Room_Occupancy_Count
8773,25.38,25.44,25.06,25.56,0,0,0,0,0.07,0.05,0.05,0.09,370,-0.042308,0,0,0
9828,25.13,25.13,24.56,25.19,0,0,0,0,0.07,0.05,0.06,0.07,345,0.000000,0,0,0
4211,25.44,25.44,25.00,25.56,0,0,0,0,0.07,0.05,0.06,0.06,375,0.000000,0,0,0
783,26.31,26.38,26.00,26.38,149,236,180,10,0.44,0.31,1.34,0.27,965,3.876923,0,0,1
5142,25.06,25.13,24.50,25.38,3,3,18,13,0.07,0.05,0.05,0.05,360,0.065385,0,0,0


## Paso 5: División del *dataset* de entrenamiento en *features* y *output*

In [68]:
# Get features columns
X = preprocess.get_features(train_df)

# Get output column
y = preprocess.get_output(train_df)

In [69]:
X.head()

,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR
894,26.31,27.94,26.06,26.44,150,239,182,11,1.65,0.34,0.10,0.09,1210,0.426923,1,0
1165,25.69,25.69,25.31,25.88,0,0,0,0,0.07,0.05,0.06,0.06,620,-2.200000,0,0
249,26.00,27.88,25.81,26.19,164,253,270,70,0.15,0.14,0.07,0.07,860,0.880769,0,1
9011,25.31,25.31,24.94,25.38,0,0,0,0,0.06,0.05,0.05,0.09,345,0.000000,0,0
1659,25.38,25.31,24.88,25.75,0,0,0,0,0.07,0.05,0.06,0.06,370,0.000000,0,0


In [70]:
y.head()

894     1
1165    0
249     1
9011    0
1659    0
Name: Room_Occupancy_Count, dtype: int32

# 2. Entrenamiento y seguimiento de la huella de carbono con **CodeCarbon**

In [74]:
!pip install eco2ai
!pip install --upgrade setuptools
!pip install --ignore-installed Apscheduler

     ---------------------------------------- 1.2/1.2 MB 5.3 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 61.2.0
    Uninstalling setuptools-61.2.0:
      Successfully uninstalled setuptools-61.2.0
  Using cached APScheduler-3.9.1-py2.py3-none-any.whl (59 kB)
  Using cached setuptools-64.0.3-py3-none-any.whl (1.2 MB)
     -------------------------------------- 500.6/500.6 kB 5.2 MB/s eta 0:00:00
  Using cached tzlocal-4.2-py3-none-any.whl (19 kB)
  Using cached pytz_deprecation_shim-0.1.0.post0-py2.py3-none-any.whl (15 kB)
  Using cached tzdata-2022.2-py2.py3-none-any.whl (336 kB)


In [75]:
import eco2ai
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

tracker = eco2ai.Tracker(project_name="TFG-Project", experiment_description="training the Logistic Regression model")

tracker.start()

lg_pipeline = Pipeline([("scaler", StandardScaler()), ("logistic_regression", LogisticRegression())])
lg_pipeline.fit(X, y, epochs=1)

tracker.stop()

C:\Users\INTEL I5\miniconda3\envs\TFG_Project\lib\site-packages\eco2ai\emission_track.py:211: UserWarning: 
    If you use a VPN, you may have problems with identifying your country by IP.
    It is recommended to disable VPN or
    manually install the ISO-Alpha-2 code of your country during initialization of the Tracker() class.
    You can find the ISO-Alpha-2 code of your country here: https://www.iban.com/country-codes
    
  warnings.warn(
C:\Users\INTEL I5\miniconda3\envs\TFG_Project\lib\site-packages\eco2ai\tools\tools_cpu.py:240: NoNeededLibrary: 
It's impossible to deretmine cpu number correctly
For now, number of cpu devices is set to 1


  warnings.warn(message="\nIt's impossible to deretmine cpu number correctly\nFor now, number of cpu devices is set to 1\n\n",
C:\Users\INTEL I5\miniconda3\envs\TFG_Project\lib\site-packages\eco2ai\tools\tools_gpu.py:35: NoGPUWarning: 

There is no any available GPU devices or your gpu is not supported by Nvidia library!
The thacker will co

LookupError: No trigger by the name "interval" was found

In [ ]:
# Logistic Regression
#training.train_LR_codecarbon(X,y)

In [ ]:
# Random Forest
#training.train_RF_codecarbon(X,y)

In [ ]:
# Linear SVC (Support Vector Machines)
#training.train_SVC_codecarbon(X,y)